In [1]:
import glob
import os, re 
import numpy as np
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['I hear you callin\', "Here I come baby"', 'To save you, oh oh', "Baby no more stallin'"]


In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

I hear you callin', "Here I come baby"
To save you, oh oh
Baby no more stallin'
These hands have been longing to touch you baby
And now that you've come around, to seein' it my way
You won't regret it baby, and you surely won't forget it baby
It's unbelieveable how your body's calling for me
I can just hear it callin' callin' for me My body's callin' for you
My body's callin' for you
My body's callin' for you


In [3]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = re.sub(r'₩[[^)]*₩]','',sentence)
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" ₩1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence <end>


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Possible nested set at position 2
  # Remove the CWD from sys.path while we load stuff.


In [4]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> i hear you callin here i come baby <end>',
 '<start> to save you oh oh <end>',
 '<start> baby no more stallin <end>',
 '<start> these hands have been longing to touch you baby <end>',
 '<start> and now that you ve come around to seein it my way <end>',
 '<start> you won t regret it baby and you surely won t forget it baby <end>',
 '<start> it s unbelieveable how your body s calling for me <end>',
 '<start> i can just hear it callin callin for me my body s callin for you <end>',
 '<start> my body s callin for you <end>',
 '<start> my body s callin for you <end>']

In [5]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2   4 184 ...   0   0   0]
 [  2   9 530 ...   0   0   0]
 [  2  48  36 ...   0   0   0]
 ...
 [  2   4  86 ...   0   0   0]
 [  2   8 153 ...   0   0   0]
 [  2 155  14 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7fd23f591f90>


In [6]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 15: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : the
6 : you
7 : and
8 : a
9 : to
10 : it
11 : me
12 : my
13 : in
14 : t
15 : s


In [7]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2   4 184   6 814  89   4  62  48   3   0   0   0   0]
[  4 184   6 814  89   4  62  48   3   0   0   0   0   0]


In [8]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE


VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [9]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [10]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-2.72382658e-05,  4.46836711e-05, -8.80919906e-05, ...,
         -5.99580635e-05,  1.01297272e-04,  1.89255108e-04],
        [-8.04440447e-07,  1.98442620e-04, -1.97466623e-04, ...,
          1.87326747e-04,  3.20137100e-04, -6.30683553e-06],
        [ 1.82463715e-04,  5.20797214e-04, -2.04654534e-05, ...,
          8.96554557e-05,  2.38384207e-04,  4.17499496e-06],
        ...,
        [ 1.02365820e-03, -4.74782195e-04,  4.02193080e-04, ...,
         -4.72841435e-04, -7.10584689e-04, -2.80030654e-04],
        [ 5.66690695e-04, -4.59059898e-04,  7.34972418e-04, ...,
         -3.49254755e-04, -4.19546326e-04, -2.62565474e-04],
        [ 7.52152919e-05, -4.28111642e-04,  1.05836592e-03, ...,
         -1.67720369e-04, -9.45101274e-05, -2.61153182e-04]],

       [[-2.72382658e-05,  4.46836711e-05, -8.80919906e-05, ...,
         -5.99580635e-05,  1.01297272e-04,  1.89255108e-04],
        [-4.16316943e-05,  8.98811122e-05, -1

In [11]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [12]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
686/686 [==============================] - 216s 311ms/step - loss: 4.0331
Epoch 2/30
686/686 [==============================] - 215s 314ms/step - loss: 3.1186
Epoch 3/30
686/686 [==============================] - 217s 316ms/step - loss: 2.9166
Epoch 4/30
686/686 [==============================] - 218s 318ms/step - loss: 2.7730
Epoch 5/30
686/686 [==============================] - 219s 318ms/step - loss: 2.6479
Epoch 6/30
686/686 [==============================] - 219s 318ms/step - loss: 2.5375
Epoch 7/30
686/686 [==============================] - 219s 319ms/step - loss: 2.4269
Epoch 8/30
686/686 [==============================] - 221s 322ms/step - loss: 2.3338
Epoch 9/30
686/686 [==============================] - 221s 322ms/step - loss: 2.2425
Epoch 10/30
686/686 [==============================] - 221s 322ms/step - loss: 2.1481
Epoch 11/30
686/686 [==============================] - 221s 321ms/step - loss: 2.0636
Epoch 12/30
686/686 [==============================] - 221s 321

In [15]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [16]:
tf.data.Dataset.from_tensor_slices().shuffle(BUFFER_SIZE)

BUFFER_SIZE = len(enc_train) 
BATCH_SIZE = 256 steps_per_epoch = len(enc_train) // BATCH_SIZE
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)) 
dataset = dataset.shuffle(BUFFER_SIZE) 
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True) dataset

SyntaxError: invalid syntax (<ipython-input-16-dcb99045be75>, line 4)

In [17]:
tf.data.Dataset.from_tensor_slices().shuffle(BUFFER_SIZE)

BUFFER_SIZE = len(enc_val) 
BATCH_SIZE = 256 steps_per_epoch = len(enc_val) // BATCH_SIZE
valset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)) 
valset = valset.shuffle(BUFFER_SIZE) 
valset = valset.batch(BATCH_SIZE, drop_remainder=True) valset

SyntaxError: invalid syntax (<ipython-input-17-2208ec8329f9>, line 4)

In [19]:
model.fit(dataset_train, validation_data=dataset_val)

NameError: name 'dataset_train' is not defined

In [20]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2   4 184   6 814  89   4  62  48   3   0   0   0   0]
[  4 184   6 814  89   4  62  48   3   0   0   0   0   0]


In [21]:
print(src_input.shape)
print(src_input.shape)

(175749, 14)
(175749, 14)


In [22]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2,shuffle=True,random_state=111)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


In [23]:
len(src_input)

175749

In [24]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE


VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [25]:
#Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [34]:
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)

NameError: name 'lyrics' is not defined